# 1. Install package and download source code/image.



In [1]:
%%capture
#@title
import os
import sys
import tensorflow.compat.v1 as tf

# Download source code.
if "efficientdet" not in os.getcwd():
  !git clone --depth 1 https://github.com/google/automl
  os.chdir('automl/efficientdet')
  sys.path.append('.')
  !pip install -r requirements.txt
  !pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
else:
  !git pull

In [ ]:
MODEL = 'efficientdet-d2'  #@param

def download(m):
  if m not in os.listdir():
    !wget https://storage.googleapis.com/cloud-tpu-checkpoints/efficientdet/coco/{m}.tar.gz
    !tar zxf {m}.tar.gz
  ckpt_path = os.path.join(os.getcwd(), m)
  return ckpt_path

# Download checkpoint.
ckpt_path = download(MODEL)
print('Use model in {}'.format(ckpt_path))

# Prepare image and visualization settings.
image_url =  'https://user-images.githubusercontent.com/11736571/77320690-099af300-6d37-11ea-9d86-24f14dc2d540.png'#@param
image_name = 'img.png' #@param
!wget {image_url} -O img.png
import os
img_path = os.path.join(os.getcwd(), 'img.png')

min_score_thresh = 0.35  #@param
max_boxes_to_draw = 200  #@param
line_thickness =   2#@param

import PIL
# Get the largest of height/width and round to 128.
image_size = max(PIL.Image.open(img_path).size)

# 2. View graph in TensorBoard

In [ ]:
!python model_inspect.py --model_name={MODEL} --logdir=logs &> /dev/null
%load_ext tensorboard
%tensorboard --logdir logs

# 3. Training EfficientDets

## 3.1 Prepare data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
dataset_dir = "tfrecord"
if dataset_dir not in os.getcwd():
    !mkdir {dataset_dir}
# !unzip /content/drive/My\ Drive/20200722/task_garbage_detection_2-2020_07_23_13_39_54-tfrecord-1.0.zip -d ./{dataset_dir}
!unzip /content/drive/My\ Drive/20200730/task_garbage_det_fps_15_img_annot_322-2020_07_24_13_19_02-tfrecord-1.0.zip -d ./{dataset_dir}

## 3.2 Train Pascal VOC 2012 from COCO checkpoint for the whole net.

In [ ]:
# generating train tfrecord is large, so we skip the execution here.
import os
if MODEL not in os.listdir():
  !wget https://storage.googleapis.com/cloud-tpu-checkpoints/efficientdet/coco/{MODEL}.tar.gz
  !tar xf {MODEL}.tar.gz

!rm /content/tmp -r
!mkdir /content/tmp; mkdir /content/tmp/model_dir/

file_pattern = 'default.tfrecord'
images_per_epoch = 322
# key option: use --ckpt rather than --backbone_ckpt.
!python main.py --mode=train_and_eval \
    --training_file_pattern=tfrecord/{file_pattern} \
    --validation_file_pattern=tfrecord/{file_pattern} \
    --model_name={MODEL} \
    --model_dir=/content/tmp/model_dir/{MODEL}-finetune \
    --ckpt={MODEL} \
    --train_batch_size=5 \
    --eval_batch_size=1 --eval_samples={images_per_epoch}  \
    --num_examples_per_epoch={images_per_epoch}  --num_epochs=15  \
    --hparams="num_classes=5,moving_average_decay=0,autoaugment_policy=v1" #,mixed_precision=true"

## 3.3 View tensorboard for loss and accuracy.


In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/tmp/model_dir/
# Notably, this is just a demo with almost zero accuracy due to very limited
# training steps, but we can see finetuning has smaller loss than training
# from scratch at the begining.

## 3.4 Inference video

In [17]:
# create classes.yaml
!echo "1: PET" > /content/classes.yaml
!echo "2: plastic" >> /content/classes.yaml
!echo "3: light_bulb" >> /content/classes.yaml
!echo "4: paper" >> /content/classes.yaml
!echo "5: cardboard" >> /content/classes.yaml

In [9]:
video_path = '/content/drive/My\ Drive/20200724/VID_20200722_115436_stabiilizo_annot.mp4'

In [ ]:
!mkdir /content/efficientdet-d0-finetune;
!unzip /content/drive/My\ Drive/20200730/best_weights_d0_160.zip -d /content/efficientdet-d0-finetune/

In [ ]:
%cd /content/automl/efficientdet/

In [ ]:
# MODEL = 'efficientdet-d0'
# ckpt_path = '/content/tmp/model_dir/efficientdet-d0-finetune/model.ckpt-35622'
# ckpt_path = '/content/efficientdet-d0-finetune/backup/model.ckpt-160'
ckpt_path = '/content/tmp/model_dir/efficientdet-d2-finetune/backup/model.ckpt-192'

saved_model_dir = '/content/saved_model'
!rm {saved_model_dir} -r
# Step 1: export model
!python model_inspect.py --runmode=saved_model \
  --model_name={MODEL} --ckpt_path={ckpt_path} \
  --saved_model_dir={saved_model_dir} \
  --min_score_thresh=0.2 \
  --hparams="num_classes=5,moving_average_decay=0" #,label_id_mapping=/content/classes.yaml,"

In [ ]:
!echo {saved_model_dir}
!cd /content/tmp/model_dir/efficientdet-d2-finetune; zip /content/best_weights_d2_192.zip ./backup/* graph.pbtxt

In [18]:
# !cp /content/best_weights_d0_160.zip /content/drive/My\ Drive/20200730/

In [ ]:
# Step 2: do inference with saved model using saved_model_video
!python model_inspect.py --runmode=saved_model_video \
  --model_name={MODEL} --ckpt_path={ckpt_path} \
  --saved_model_dir={saved_model_dir} \
  --min_score_thresh=0.5 \
  --input_video={video_path} --output_video=/content/output2.mov \
  --hparams="image_size=1920x1080,num_classes=5,moving_average_decay=0,label_id_mapping=/content/classes.yaml,"